<a href="https://colab.research.google.com/github/francescobarbara/BGAIN/blob/main/BGAIN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/francescobarbara/GAIN

Cloning into 'GAIN'...
remote: Enumerating objects: 80, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 80 (delta 8), reused 0 (delta 0), pack-reused 58
Unpacking objects: 100% (80/80), done.


Utils

In [ ]:
from tqdm import tqdm

In [ ]:
import numpy as np
#import tensorflow as tf
##IF USING TF 2 use following import to still use TF < 2.0 Functionalities
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()


def normalization (data, parameters=None):    #tutto ok, input semplice dataset
  '''Normalize data 
  
  Args:
    - data: original data
  
  Returns:
    - norm_data: normalized data
    - norm_parameters: min_val, max_val for each feature for renormalization
  '''

  # Parameters
  _, dim = data.shape
  norm_data = data.copy()
  
  if parameters is None:
  
    # MixMax normalization
    mean = np.nanmean(data, axis = 0)
    std = np.nanstd(data, axis = 0)
    
    # For each dimension
    for i in range(dim):
        
      norm_data[:,i] = norm_data[:,i] - mean[i]
      if std[i] != 0:
          norm_data[:,i] = norm_data[:,i] / std[i]   
      
    # Return norm_parameters for renormalization
    norm_parameters = {'mean': mean,
                       'std': std}  
      
  return norm_data, norm_parameters

def renormalization (norm_data, norm_parameters): #norm_pars is a dict
  '''Renormalize data from [0, 1] range to the original range.
  
  Args:
    - norm_data: normalized data
    - norm_parameters: min_val, max_val for each feature for renormalization
  
  Returns:
    - renorm_data: renormalized original data
  '''
  
  mean = norm_parameters['mean']
  std = norm_parameters['std']

  _, dim = norm_data.shape
  renorm_data = norm_data.copy()
    
  for i in range(dim):
    renorm_data[:,i] = renorm_data[:,i] * std[i] + mean[i]   
    
  return renorm_data


def rounding (imputed_data, data_x):
  '''Round imputed data for categorical variables.
  
  Args:
    - imputed_data: imputed data
    - data_x: original data with missing values
    
  Returns:
    - rounded_data: rounded imputed data
  '''
  
  _, dim = data_x.shape
  rounded_data = imputed_data.copy()
  
  for i in range(dim):
    temp = data_x[~np.isnan(data_x[:, i]), i]   
    # Only for the categorical variable
    if len(np.unique(temp)) < 20:
      rounded_data[:, i] = np.round(rounded_data[:, i])
      
  return rounded_data


def rmse_loss (ori_data, imputed_data, data_m):   #just takes two datasets as inputs
  '''Compute RMSE loss between ori_data and imputed_data
  
  Args:
    - ori_data: original data without missing values
    - imputed_data: imputed data
    - data_m: indicator matrix for missingness
    
  Returns:
    - rmse: Root Mean Squared Error
  '''
  
  ori_data, norm_parameters = normalization(ori_data)
  imputed_data, _ = normalization(imputed_data, norm_parameters)
    
  # Only for missing values
  nominator = np.sum(((1-data_m) * ori_data - (1-data_m) * imputed_data)**2)
  denominator = np.sum(1-data_m)
  
  rmse = np.sqrt(nominator/float(denominator))
  
  return rmse


def xavier_init(size):
  '''Xavier initialization.
  
  Args:
    - size: vector size
    
  Returns:
    - initialized random vector.
  '''
  in_dim = size[0]
  xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
  return tf.random_normal(shape = size, stddev = xavier_stddev)
      

def binary_sampler(p, rows, cols):  #creates matrix with entries Bernoulli p
  '''Sample binary random variables.
  
  Args:
    - p: probability of 1
    - rows: the number of rows
    - cols: the number of columns
    
  Returns:
    - binary_random_matrix: generated binary random matrix.
  '''
  unif_random_matrix = np.random.uniform(0., 1., size = [rows, cols])
  binary_random_matrix = 1*(unif_random_matrix < p)
  return binary_random_matrix


def uniform_sampler(low, high, rows, cols):
  '''Sample uniform random variables.
  
  Args:
    - low: low limit
    - high: high limit
    - rows: the number of rows
    - cols: the number of columns
    
  Returns:
    - uniform_random_matrix: generated uniform random matrix.
  '''
  return np.random.uniform(low, high, size = [rows, cols])       


def sample_batch_index(total, batch_size): #returns a number batch size of indices
  '''Sample index of the mini-batch.
  
  Args:
    - total: total number of samples
    - batch_size: batch size
    
  Returns:
    - batch_idx: batch index
  '''
  total_idx = np.random.permutation(total)
  batch_idx = total_idx[:batch_size]
  return batch_idx





Instructions for updating:
non-resource variables are not supported in the long term


New 3 methods here

In [ ]:
def create_dataset(n,d):
  return np.random.uniform(size = (n,d))

In [ ]:
def create_sigma_1 (data_x, data_m): 
    
    #data_m = np.array(data_m)
    #data should already have been centered IMPORTANT
    n, d = data_x.shape
    sigma_1 = np.zeros((d, d))
    
    
    
    for i in range(d):
        for j in range(d):
            count = 0
            for row in range(n):
                if data_m[row, i] == 1 and data_m[row, j] == 1:
                    sigma_1[i,j] += data_x[row, i] * data_x[row, j]
                    count += 1
                    
            if count > 1:
                sigma_1[i,j] = sigma_1[i,j]/(count - 1)   #for unbiasedness
    print('goof')            
    return sigma_1

In [ ]:
def conditional_sigma2(m, sigma0, sigma1, d):
    #return (A) at page D, but for all i grouped together
    '''print(m)
    print(type(m))
    print(m.shape)
    print(type(m.shape))'''
    


    out = np.zeros(d)
    j_indices = m == 1
    print(j_indices)
    print(type(j_indices))
    sigma1_jj_inv = np.linalg.inv(sigma1[j_indices][:, j_indices])
    
    for i in range(d):
        
        sigma0_i_j = sigma0[i, j_indices].reshape(1, sum(j_indices))
        
        out[i] = sigma0[i,i] - np.matmul( np.matmul(sigma0_i_j, sigma1_jj_inv) , np.transpose(sigma0_i_j))
    print('good')  
    return out

In [ ]:
def conditional_mu2(m, sigma0, sigma1, d):
    #return (..) in (B) at page D it returns a list of 1xsum(m) vectors
    #m = np.array(m) #doesn't work
    ''' print(m)
    print(type(m))
    print(m.shape)
    print(type(m.shape))'''
    


    out = np.zeros((d, d))
    j_indices = (m == 1)
    
    sigma1_jj_inv = np.linalg.inv(sigma1[j_indices][:, j_indices])
    
    for i in range(d):
        
        sigma0_i_j = sigma0[i, j_indices].reshape(1, sum(j_indices))
        
        out[i] = np.matmul(sigma0_i_j, sigma1_jj_inv)
    print('goog')    
    return out

In [ ]:
def matmul(tensor1, tensor2):
  tensor1 = tf.cast(tensor1, dtype='float64')
  tensor2 = tf.cast(tensor2, dtype='float64')
  return tf.matmul(tensor1, tensor2)

Main thing

In [ ]:
def gain (data_x, gain_parameters, priors):
  '''Impute missing values in data_x
  
  Args:
    - data_x: original data with missing values
    - gain_parameters: GAIN network parameters:     #this is a dictionary
      - batch_size: Batch size
      - hint_rate: Hint rate
      - alpha: Hyperparameter
      - iterations: Iterations
      
  Returns:
    - imputed_data: imputed data
  '''
  ''' priors:
      priors['covariance'] is a dxd array
      priors['mean'] is a d array
      '''
  
  

  
  # Define mask matrix
  data_m = 1-np.isnan(data_x)
   
  
  
  
  # System parameters
  batch_size = gain_parameters['batch_size']
  hint_rate = gain_parameters['hint_rate']
  alpha = gain_parameters['alpha']
  iterations = gain_parameters['iterations']
  
  # Other parameters
  no, dim = data_x.shape
  
  # Hidden state dimensions
  h_dim = int(dim)
  
  # Normalization
  norm_data, norm_parameters = normalization(data_x)
  norm_data_x = np.nan_to_num(norm_data, 0)
  
  #NEW STUFF HERE
  sigma1 = create_sigma_1(norm_data, data_m) 
  mu1 = np.nanmean(data_x, axis = 0)  #should be all zero (sanity check)
  #ALSO, VOLENDO, MODIFY PRIORS USING NORM_PARAMETERS
  sigma0 = priors['covariance']
  mu0 = priors['mean']
  
  #CONVERT THEM TO TENSORS
  sigma1 = tf.cast(tf.convert_to_tensor(sigma1), dtype = 'float32')
  mu1 = tf.cast(tf.convert_to_tensor(mu1), dtype = 'float32')
  sigma0 = tf.cast(tf.convert_to_tensor(sigma0), dtype = 'float32')
  mu0 = tf.cast(tf.convert_to_tensor(mu0), dtype = 'float32')


  ## GAIN architecture   
  # Input placeholders
  # Data vector
  X = tf.placeholder(tf.float64, shape = [None, dim])     #changed to 64 !!!!!!!!!!!!
  # Mask vector 
  M = tf.placeholder(tf.float64, shape = [None, dim])
  # Hint vector
  H = tf.placeholder(tf.float64, shape = [None, dim])
  
  # Discriminator variables
  D_W1 = tf.Variable(xavier_init([dim*2, h_dim])) # Data + Hint as inputs   #credo x_tilde, h as in Algo 1
  D_b1 = tf.Variable(tf.zeros(shape = [h_dim]))
  
  D_W2 =  tf.Variable(xavier_init([h_dim, h_dim]))
  D_b2 = tf.Variable(tf.zeros(shape = [h_dim]))
  
  D_W3 =  tf.Variable(xavier_init([h_dim, dim]))
  D_b3 = tf.Variable(tf.zeros(shape = [dim]))  # Multi-variate outputs   #the output dimension should be the same as the data (we are trying to replicate it)
  
  theta_D = [D_W1, D_W2, D_W3, D_b1, D_b2, D_b3]   #set of params
  
  #Generator variables
  # Data + Mask as inputs (Random noise is in missing components)
  G_W1 = tf.Variable(xavier_init([dim*2, h_dim]))
  G_b1 = tf.Variable(tf.zeros(shape = [h_dim]))
  
  G_W2 = tf.Variable(xavier_init([h_dim, h_dim]))
  G_b2 =  tf.Variable(tf.zeros(shape = [h_dim]))
  
  G_W3 =  tf.Variable(xavier_init([h_dim, dim]))
  G_b3 = tf.Variable(tf.zeros(shape = [dim]))
  
  theta_G = [G_W1, G_W2, G_W3, G_b1, G_b2, G_b3]
  
  ## GAIN functions
  # Generator
  def generator(x,m):
    # Concatenate Mask and Data
    print(type(x))
    print(type(m))
    inputs = tf.cast(tf.concat(values = [x, m], axis = 1), dtype = 'float32')
    print(type(inputs))
    G_h1 = tf.nn.relu(tf.matmul(inputs, G_W1) + G_b1)
    G_h2 = tf.nn.relu(tf.matmul(G_h1, G_W2) + G_b2)   
    # MinMax normalized output
    G_prob = tf.nn.sigmoid(tf.matmul(G_h2, G_W3) + G_b3)      #cambia qui!!! tipo elimina sigmoid
    return G_prob
      
  # Discriminator
  def discriminator(x, h):
    # Concatenate Data and Hint
    print(type(x))
    
    inputs = tf.cast(tf.concat(values = [tf.cast(x, dtype = 'float32'), tf.cast(h, dtype = 'float32')], axis = 1), dtype = 'float32')
    print(type(inputs))
    D_h1 = tf.nn.relu(tf.matmul(inputs, D_W1) + D_b1)  
    D_h2 = tf.nn.relu(tf.matmul(D_h1, D_W2) + D_b2)
    D_logit = tf.matmul(D_h2, D_W3) + D_b3
    D_prob = tf.nn.sigmoid(D_logit)
    return D_prob
  
  def conditional(X, M, sigma0, sigma1, mu0, mu1):
    M_mat = tf.concat(values = [M for i in range(dim)], axis = 1) 
    M_mat = tf.reshape(M_mat,  shape = [tf.shape(M_mat)[0], dim, dim])
    M_mat = tf.cast(M_mat, dtype = 'float32')

    muc = mu0 + tf.reshape( tf.matmul( tf.math.multiply( tf.cast(M_mat, dtype = 'float32'), tf.cast(sigma0, dtype = 'float32')), \
                          tf.matmul(tf.cast(tf.linalg.inv(sigma1), dtype = 'float32') , tf.cast(tf.transpose(X), dtype = 'float32'))) , shape = [tf.shape(M_mat)[0], dim])
    #sigmac = sigma_diag - tf.reshape( tf.matmul( tf.math.multiply(M_mat, sigma0), tf.matmul(tf.linalg.inv(sigma1) , tf.transpose(matmul( tf.math.multiply(M_mat, sigma0)))) , shape = [tf.shape(M_mat)[0], dim])


    #calculating sigmac here
    temp = tf.matmul(  tf.math.multiply(M_mat, sigma0)    ,  tf.matmul(tf.linalg.inv(sigma1) , tf.transpose( tf.math.multiply(M_mat, sigma0) ) ) )
    temp = tf.transpose(temp)
    temp = sigma0 - temp

    indices = [[i, i] for i in range(dim)]
    temp = tf.gather_nd(tf.transpose(temp), indices, batch_dims=0, name=None)
    temp = tf.transpose(temp)
    sigmac = temp
    
    return (muc, sigmac)
      
      
  
  ## GAIN structure
  # Generator
  G_sample = generator(X, M)
  print(G_sample)
  print(type(G_sample))
 
  # Combine with observed data
  Hat_X = tf.cast(X, dtype = 'float32') * tf.cast(M, dtype = 'float32') + tf.cast(G_sample, dtype = 'float32') * tf.cast((1-M), dtype = 'float32')
  print('hat')
  print(Hat_X)
  print(type(Hat_X))
  #new here the vectors muc and sigmac
  conditional_mu, conditional_sigma = conditional(X, M, sigma0, sigma1, mu0, mu1)   #should put X instead of Hat_X
  print('cond_mu')
  print(conditional_mu)
  print(type(conditional_mu))
  
  
  
  # Discriminator
  D_prob = discriminator(Hat_X, H)
  
  ## GAIN loss
  D_loss_temp = -tf.reduce_mean(tf.cast(M, dtype = 'float32') * tf.cast(tf.log(D_prob + 1e-8), dtype = 'float32') \
                                + tf.cast((1-M), dtype = 'float32') * tf.cast(tf.log(1. - D_prob + 1e-8) , dtype = 'float32'))
  
  G_loss_temp = -tf.reduce_mean(tf.cast((1-M), dtype = 'float32') * tf.cast(tf.log(D_prob + 1e-8), dtype = 'float32'))
  
  MSE_loss = \
  tf.reduce_mean((tf.cast(M, dtype = 'float32') * tf.cast(X, dtype = 'float32') - tf.cast(M, dtype = 'float32') * tf.cast(G_sample, dtype = 'float32'))**2) / \
  tf.cast(tf.reduce_mean(M), dtype = 'float32')
  
  #ADD THIRD LOSS HERE
  print('M')
  print(M.shape)
  print('Hat_X')
  print(Hat_X.shape)
  print('conditional_mu')
  print(conditional_mu.shape)
  print('conditional_sigma')
  print(conditional_sigma.shape)
  print('D_prob')
  print(D_prob.shape)
  
  loss_3 = \
  -tf.reduce_mean( tf.cast((1-M), dtype = 'float32') * tf.cast( tf.exp(  - (tf.cast(Hat_X, dtype = 'float32') - tf.cast(conditional_mu, dtype = 'float32'))**2 \
                  / (2*tf.cast(conditional_sigma, dtype = 'float32'))) , dtype = 'float32')  )
  print('loss_3')
  print(loss_3.shape)

  """
  j_index = M == 1
  loss_3 = 0"""
  """for i in range(dim):
      loss_3 += (1 - M[i])*tf.math.exp(-tf.math.pow( (Hat_X[i] - mu0[i] - 
                tf.tensordot( tf.convert_to_tensor(conditional_mu[i]), 
                    tf.boolean_mask(Hat_X[i], j_index) - tf.boolean_mask(mu1, j_index) ) ), 2) / (2*conditional_sigma[i])) 
        """
  
  
  D_loss = D_loss_temp
  G_loss = G_loss_temp + alpha * MSE_loss  + loss_3   #as defined in the paper, add beta
  
  ## GAIN solver
  D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list=theta_D)
  G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list=theta_G)
  
  ## Iterations
  sess = tf.Session()
  sess.run(tf.global_variables_initializer())
   
  # Start Iterations
  for it in tqdm(range(iterations)):    
      
    # Sample batch
    batch_idx = sample_batch_index(no, batch_size)
    X_mb = norm_data_x[batch_idx, :]  
    M_mb = data_m[batch_idx, :]     #tells you where data is missing (look top of script)
    # Sample random vectors  
    Z_mb = uniform_sampler(0, 0.01, batch_size, dim) 
    # Sample hint vectors
    H_mb_temp = binary_sampler(hint_rate, batch_size, dim)
    H_mb = M_mb * H_mb_temp
    print('checkpoint 1')  
    # Combine random vectors with observed vectors
    X_mb = M_mb * X_mb + (1-M_mb) * Z_mb 
    print('checkpoint 2')  
    _, D_loss_curr = sess.run([D_solver, D_loss_temp], 
                              feed_dict = {M: M_mb, X: X_mb, H: H_mb})
    print('checkpoint 3') 
    _, G_loss_curr, MSE_loss_curr = \
    sess.run([G_solver, G_loss_temp, MSE_loss],
             feed_dict = {X: X_mb, M: M_mb, H: H_mb})
    print('checkpoint 4')        
  ## Return imputed data      
  Z_mb = uniform_sampler(0, 0.01, no, dim) 
  M_mb = data_m
  X_mb = norm_data_x          
  X_mb = M_mb * X_mb + (1-M_mb) * Z_mb 
  print('checkpoint 5')     
  imputed_data = sess.run([G_sample], feed_dict = {X: X_mb, M: M_mb})[0]
  print('checkpoint 6') 
  imputed_data = data_m * norm_data_x + (1-data_m) * imputed_data
  print('checkpoint 7') 
  # Renormalization
  imputed_data = renormalization(imputed_data, norm_parameters)  
  print('checkpoint 8') 
  # Rounding
  imputed_data = rounding(imputed_data, data_x)  
  print('checkpoint 9')        
  return imputed_data

Data Loader

In [ ]:
from keras.datasets import mnist


def data_loader (data_name, miss_rate):
  '''Loads datasets and introduce missingness.
  
  Args:
    - data_name: letter, spam, or mnist
    - miss_rate: the probability of missing components
    
  Returns:
    data_x: original data
    miss_data_x: data with missing values
    data_m: indicator matrix for missing components
  '''
  
  # Load data
  if data_name in ['letter', 'spam']:               #change here if you add more datasets
    file_name = '/content/GAIN/data/'+data_name+'.csv'
    data_x = np.loadtxt(file_name, delimiter=",", skiprows=1)
  elif data_name == 'mnist':
    (data_x, _), _ = mnist.load_data()
    data_x = np.reshape(np.asarray(data_x), [60000, 28*28]).astype(float)
  elif data_name == 'synthetic':
    data_x = create_dataset(n = 10000, d = 10).astype(float)
  # Parameters
  no, dim = data_x.shape
  
  # Introduce missing data
  data_m = binary_sampler(1-miss_rate, no, dim)
  miss_data_x = data_x.copy()
  miss_data_x[data_m == 0] = np.nan
      
  return data_x, miss_data_x, data_m

Main letter spam

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse

In [ ]:
def main (args):
  '''Main function for UCI letter and spam datasets.
  
  Args:
    - data_name: letter or spam
    - miss_rate: probability of missing components
    - batch:size: batch size
    - hint_rate: hint rate
    - alpha: hyperparameter
    - iterations: iterations
    
  Returns:
    - imputed_data_x: imputed data
    - rmse: Root Mean Squared Error
  '''
  
  data_name = args['data_name']
  miss_rate = args['miss_rate']
  
  gain_parameters = {'batch_size': args['batch_size'],
                     'hint_rate': args['hint_rate'],
                     'alpha': args['alpha'],
                     'iterations': args['iterations']}
  
  # Load data and introduce missingness
  ori_data_x, miss_data_x, data_m = data_loader(data_name, miss_rate)    #CHANGE HERE FOR MY EXPERIMENTS
  
  #nNEW HERE
  prior0 = np.zeros(ori_data_x.shape[1])
  prior1 = np.identity(ori_data_x.shape[1])
  priors = {'mean' : prior0, 'covariance' : prior1}
  # Impute missing data
  imputed_data_x = gain(miss_data_x, gain_parameters, priors)
  
  # Report the RMSE performance
  rmse = rmse_loss (ori_data_x, imputed_data_x, data_m)
  
  print()
  print('RMSE Performance: ' + str(np.round(rmse, 4)))
  
  return imputed_data_x, rmse


In [ ]:
args = {'data_name': 'synthetic', 'miss_rate' : 0.2, 'batch_size' : 128, 'hint_rate' : 0.9, 'alpha' : 100, 'iterations' : 10}

In [ ]:
main(args)

goof
<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>
Tensor("Sigmoid_32:0", shape=(?, 10), dtype=float32)
<class 'tensorflow.python.framework.ops.Tensor'>
hat
Tensor("add_207:0", shape=(?, 10), dtype=float32)
<class 'tensorflow.python.framework.ops.Tensor'>
cond_mu
Tensor("add_208:0", shape=(?, 10), dtype=float32)
<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>
M
(?, 10)
Hat_X
(?, 10)
conditional_mu
(?, 10)
conditional_sigma
(?, 10)
D_prob
(?, 10)
loss_3
()


  0%|          | 0/10 [00:00<?, ?it/s]

checkpoint 1
checkpoint 2
checkpoint 3


  0%|          | 0/10 [00:00<?, ?it/s]


InvalidArgumentError: ignored